<a href="https://colab.research.google.com/github/FernandoSimon22/vision_artificial/blob/main/YOLOv5m_COCOevaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone GitHub repository, install dependencies and check PyTorch and GPU.

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install
%pip install -q roboflow

import torch
import utils
import os
from IPython.display import Image, clear_output
display = utils.notebook_init()  # checks


YOLOv5 🚀 v7.0-423-g567c6646 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.1/235.7 GB disk)


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lX9sE2RPQOCv2R7cn0io")
project = rf.workspace("jota22").project("am_boundingbox")
version = project.version(22)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to AM_BoundingBox-22 in yolov5pytorch:: 100%|██████████| 462/462 [00:00<00:00, 7087.41it/s]


In [3]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

### Train YOLOv5s

In [4]:
# Train YOLOv5s from scracht for 300 epochs
!python train.py --img 768 --batch 8 --epochs 200 --data /content/yolov5/AM_BoundingBox-22/data.yaml --weights yolov5m.pt --cache


Se han truncado las últimas 5000 líneas del flujo de salida.
      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
  0% 0/23 [00:00<?, ?it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     84/199       6.1G    0.04697     0.2063          0        713        768:   4% 1/23 [00:00<00:06,  3.53it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     84/199       6.1G    0.04664     0.2151          0        764        768:   9% 2/23 [00:00<00:06,  3.46it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     84/199       6.1G    0.04639     0

In [5]:
# evaluation YOLOv5s with test set
!python val.py \
  --data /content/yolov5/AM_BoundingBox-22/data.yaml \
  --task test \
  --weights /content/yolov5/runs/train/exp/weights/best.pt \
  --img 768 \
  --batch 8

val: data=/content/yolov5/AM_BoundingBox-22/data.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=8, imgsz=768, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-423-g567c6646 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Traceback (most recent call last):
  File "/content/yolov5/utils/dataloaders.py", line 587, in __init__
    raise FileNotFoundError(f"{prefix}{p} does not exist")
FileNotFoundError: test: /content/yolov5/test/images does not exist

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/yolov5/val.py", line 604, in <module>
    main(opt)
  File "/cont

# EVALUAR CON COCOevaluator - DATASET EN FORMATO COCO

In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lX9sE2RPQOCv2R7cn0io")
project = rf.workspace("jota22").project("am_boundingbox")
version = project.version(22)
dataset = version.download("coco")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/datasets/AM_BoundingBox-22 in coco:: 100%|██████████| 233/233 [00:00<00:00, 5627.77it/s]


In [12]:
# ==========================================
# EVALUACIÓN COCO DE YOLOv5n (coco_evaluator)
# ==========================================
# Requisitos:
# - Pesos YOLOv5 (.pt) entrenados por ti
# - Conjunto de validación en formato COCO (imágenes + json)
# - pycocotools
# - Repo YOLOv5 (torch.hub lo descarga si no está)
# ------------------------------------------

!pip -q install pycocotools

import os, json, glob, torch
from tqdm import tqdm
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# -----------------------------
# 1) RUTAS / PARÁMETROS
# -----------------------------
WEIGHTS = "/content/yolov5/runs/train/exp/weights/best.pt"                # <-- tu .pt entrenado
IMG_DIR = "/content/datasets/AM_BoundingBox-22/valid"             # <-- carpeta con imágenes val
GT_COCO_JSON = "/content/datasets/AM_BoundingBox-22/valid/_annotations.coco.json"  # GT
OUT_JSON = "/content/yolov5n_preds_coco.json"       # dónde guardar predicciones

CONF_THRES = 0.001    # deja bajo para que COCOeval aplique el barrido de scores
IOU_THRES = 0.65      # NMS interna de YOLOv5
IMG_SIZE = 768
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
# -----------------------------
# 2) CARGAR MODELO YOLOv5n
# -----------------------------
# Usamos torch.hub para cargar el repo oficial de YOLOv5
# Nota: 'custom' permite cargar pesos propios.
model = torch.hub.load("ultralytics/yolov5", "custom",
                       path=WEIGHTS, force_reload=False)
model.conf = CONF_THRES
model.iou = IOU_THRES
model.max_det = 300
model.imgsz = IMG_SIZE
model.to(DEVICE).eval()

# Nombres de clases del modelo (idx -> name)
yolo_names = model.names  # dict o list
if isinstance(yolo_names, list):
    yolo_names = {i: n for i, n in enumerate(yolo_names)}

print("Clases del modelo:", yolo_names)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-423-g567c6646 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


Clases del modelo: {0: 'def'}


In [14]:
# -----------------------------
# 3) MAPEO DE CLASES -> COCO
# -----------------------------
coco_gt = COCO(GT_COCO_JSON)

# name -> cat_id en tu GT COCO
name_to_coco = {c["name"]: c["id"] for c in coco_gt.loadCats(coco_gt.getCatIds())}
print("Clases en COCO GT:", name_to_coco)

# Verificación rápida de correspondencia de nombres
missing = [n for n in yolo_names.values() if n not in name_to_coco]
if missing:
    raise ValueError(
        f"❌ Estas clases del modelo NO existen en tu COCO GT: {missing}\n"
        "Asegúrate de que los nombres coincidan exactamente."
    )


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Clases en COCO GT: {'DEFECTOS': 0, 'def': 1}


In [15]:
# -----------------------------
# 4) INFERENCIA Y JSON DE COCO
# -----------------------------
# COCOeval espera una lista con objetos {image_id, category_id, bbox, score}
# bbox en formato [x, y, w, h] en pixeles absolutos
results = []
img_paths = sorted(
    [p for ext in ("*.jpg", "*.jpeg", "*.png") for p in glob.glob(os.path.join(IMG_DIR, ext))]
)

# Prepara un mapeo ruta->image_id leyendo el GT (seguro contra desórdenes)
# COCO guarda file_name relativo. Lo normal es que coincida con tu archivo en IMG_DIR.
id_by_filename = {img["file_name"]: img["id"] for img in coco_gt.loadImgs(coco_gt.getImgIds())}

for im_path in tqdm(img_paths, desc="Inferencia YOLOv5n"):
    # Obtener image_id desde el nombre de archivo
    file_name = os.path.basename(im_path)
    if file_name not in id_by_filename:
        # si el JSON tiene subcarpetas en file_name, intenta una coincidencia flexible
        # (puedes ajustar a tu caso)
        candidates = [k for k in id_by_filename.keys() if k.endswith(file_name)]
        if not candidates:
            print(f"⚠️ No se encontró image_id para {file_name}; se omite.")
            continue
        file_name = candidates[0]

    image_id = int(id_by_filename[file_name])

    # correr inferencia
    results_yolo = model(im_path, size=IMG_SIZE)
    preds = results_yolo.xyxy[0].detach().cpu().numpy()  # [N,6]: x1,y1,x2,y2,conf,cls

    for x1, y1, x2, y2, conf, cls_idx in preds:
        cls_idx = int(cls_idx)
        cls_name = yolo_names[cls_idx]
        category_id = int(name_to_coco[cls_name])

        w, h = x2 - x1, y2 - y1
        # descartar cajas degeneradas
        if w <= 0 or h <= 0:
            continue

        results.append({
            "image_id": image_id,
            "category_id": category_id,
            "bbox": [float(x1), float(y1), float(w), float(h)],
            "score": float(conf)
        })

# Guardar predicciones
with open(OUT_JSON, "w") as f:
    json.dump(results, f)
print(f"✅ Predicciones guardadas en: {OUT_JSON} (total: {len(results)})")



Inferencia YOLOv5n:   0%|          | 0/23 [00:00<?, ?it/s]`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
Inferencia YOLOv5n:   9%|▊         | 2/23 [00:00<00:01, 15.16it/s]`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
Inferencia YOLOv5n:  17%|█▋        | 4/23 [00:00<00:01, 16.34it/s]`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
Inferencia YOLOv5n:  30%|███       | 7/23 [00:00<00:00

✅ Predicciones guardadas en: /content/yolov5n_preds_coco.json (total: 6900)


In [16]:
# -----------------------------
# 5) EVALUAR CON COCOeval
# -----------------------------
coco_dt = coco_gt.loadRes(OUT_JSON)
coco_eval = COCOeval(coco_gt, coco_dt, iouType="bbox")

# (opcional) ajusta maxDets para asemejar a Ultralytics
coco_eval.params.maxDets = [1, 10, 300]

coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Métricas clave comparables a Ultralytics
print({
    "mAP50-95": coco_eval.stats[0],  # AP@[.50:.95]
    "mAP50":    coco_eval.stats[1],  # AP@0.50
    "AR@100":   coco_eval.stats[8],  # AR @[.50:.95], area=all, maxDets=100 (índice 8)
})


Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.82s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=300 ] = 0.661
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=300 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=300 ] = 0.275
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=300 ] = 0.210
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=300 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.014
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.104
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.455
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=

# División en 5 folds

In [ ]:
import os
import shutil
import random
from sklearn.model_selection import KFold
from pathlib import Path

# Configura tu ruta de dataset aquí
DATASET_PATH = "/content/yolov5/AM_BoundingBox-22/train"
IMAGES_DIR = os.path.join(DATASET_PATH, "images")
LABELS_DIR = os.path.join(DATASET_PATH, "labels")

# Número de folds
K_FOLDS = 5

# Lista de todas las imágenes y sus etiquetas
images = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(".jpg") or f.endswith(".png")])
labels = sorted([f for f in os.listdir(LABELS_DIR) if f.endswith(".txt")])

# Comprobación
assert len(images) == len(labels), "Número de imágenes y etiquetas no coincide"

# Crear los folds
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(images)):
    print(f"Preparando Fold {fold + 1}")

    fold_dir = f"/content/folds/fold{fold + 1}"
    train_img_dir = os.path.join(fold_dir, "train/images")
    train_lbl_dir = os.path.join(fold_dir, "train/labels")
    val_img_dir = os.path.join(fold_dir, "val/images")
    val_lbl_dir = os.path.join(fold_dir, "val/labels")

    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(train_lbl_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(val_lbl_dir, exist_ok=True)

    for i in train_idx:
        shutil.copy(os.path.join(IMAGES_DIR, images[i]), os.path.join(train_img_dir, images[i]))
        shutil.copy(os.path.join(LABELS_DIR, labels[i]), os.path.join(train_lbl_dir, labels[i]))

    for i in val_idx:
        shutil.copy(os.path.join(IMAGES_DIR, images[i]), os.path.join(val_img_dir, images[i]))
        shutil.copy(os.path.join(LABELS_DIR, labels[i]), os.path.join(val_lbl_dir, labels[i]))

print("✅ División en 5 folds completada")

Preparando Fold 1
Preparando Fold 2
Preparando Fold 3
Preparando Fold 4
Preparando Fold 5
✅ División en 5 folds completada


# Validación cruzada (5-fold CV) con YOLOv5 y calcular las métricas promedio.

# ✅ 1. Configuración inicial

In [ ]:
import os
import shutil
from sklearn.model_selection import KFold
import glob
from pathlib import Path
import yaml

# Rutas
IMAGES_DIR = '/content/yolov5/AM_BoundingBox-22/train/images'
LABELS_DIR = '/content/yolov5/AM_BoundingBox-22/train/labels'
FOLDS_DIR = '/content/folds'
os.makedirs(FOLDS_DIR, exist_ok=True)

# Archivos
image_files = sorted(glob.glob(os.path.join(IMAGES_DIR, '*.jpg')))
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ✅ 2. Crear carpetas para cada fold

In [ ]:
for fold, (train_idx, val_idx) in enumerate(kf.split(image_files), 1):
    fold_path = os.path.join(FOLDS_DIR, f'fold{fold}')
    train_img = os.path.join(fold_path, 'images', 'train')
    val_img = os.path.join(fold_path, 'images', 'val')
    train_lbl = os.path.join(fold_path, 'labels', 'train')
    val_lbl = os.path.join(fold_path, 'labels', 'val')
    for d in [train_img, val_img, train_lbl, val_lbl]: os.makedirs(d, exist_ok=True)

    for i in train_idx:
        img = image_files[i]
        lbl = os.path.join(LABELS_DIR, Path(img).stem + '.txt')
        shutil.copy(img, train_img)
        shutil.copy(lbl, train_lbl)

    for i in val_idx:
        img = image_files[i]
        lbl = os.path.join(LABELS_DIR, Path(img).stem + '.txt')
        shutil.copy(img, val_img)
        shutil.copy(lbl, val_lbl)

# ✅ 3. Ejecutar entrenamiento en cada fold

In [ ]:
import subprocess

results = []

for fold in range(1, 6):
    fold_path = f'{FOLDS_DIR}/fold{fold}'

    # Crear YAML
    data_yaml = {
        'train': os.path.join(fold_path, 'images', 'train'),
        'val': os.path.join(fold_path, 'images', 'val'),
        'nc': 1,
        'names': ['def']
    }
    yaml_path = os.path.join(fold_path, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print(f'🔁 Entrenando fold {fold}...')

    result = subprocess.run([
        'python', 'train.py',
        '--img', '768',
        '--batch', '8',
        '--epochs', '250',
        '--data', yaml_path,
        '--weights', 'yolov5s.pt',
        '--project', '/content/folds/results',
        '--name', f'fold{fold}',
        '--exist-ok'
    ])
    results.append(result)

🔁 Entrenando fold 1...
🔁 Entrenando fold 2...
🔁 Entrenando fold 3...
🔁 Entrenando fold 4...
🔁 Entrenando fold 5...


# ✅ 4. Promediar métricas de rendimiento (mAP, precisión, recall)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/folds/results/fold1/results.csv')
print(df.columns.tolist())


['               epoch', '      train/box_loss', '      train/obj_loss', '      train/cls_loss', '   metrics/precision', '      metrics/recall', '     metrics/mAP_0.5', 'metrics/mAP_0.5:0.95', '        val/box_loss', '        val/obj_loss', '        val/cls_loss', '               x/lr0', '               x/lr1', '               x/lr2']


In [ ]:
import pandas as pd

maps = []
precisions = []
recalls = []

for fold in range(1, 6):
    path = f'/content/folds/results/fold{fold}/results.csv'
    df = pd.read_csv(path)
    last_row = df.iloc[-1]

    # Asignación segura por nombre parcial de columnas
    mAP_col = [col for col in df.columns if 'mAP_0.5' in col][0]
    prec_col = [col for col in df.columns if 'precision' in col][0]
    recall_col = [col for col in df.columns if 'recall' in col][0]

    maps.append(float(last_row[mAP_col]))
    precisions.append(float(last_row[prec_col]))
    recalls.append(float(last_row[recall_col]))

print(f'📊 Promedio mAP@0.5: {sum(maps)/5:.4f}')
print(f'📊 Promedio precisión: {sum(precisions)/5:.4f}')
print(f'📊 Promedio recall: {sum(recalls)/5:.4f}')


📊 Promedio mAP@0.5: 0.5796
📊 Promedio precisión: 0.6254
📊 Promedio recall: 0.5823


In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)  # or yolov5n - yolov5x6 or custom
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-418-ga493afe1 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
image 1/1: 720x1280 2 persons, 1 tie, 1 cell phone
Speed: 1054.5ms pre-process, 43.6ms inference, 140.4ms NMS per image at shape (1, 3, 384, 640)
